$$ loss(\mathbf{x},\operatorname{class} ) = \log \left(\frac{\exp (\mathbf{x}[\operatorname{class}])}{\sum_{j} \exp (\mathbf{x}[j])}\right)=-\mathbf{x}[\text { class }]+\log \left(\sum_{j} \exp (\mathbf{x}[j])\right) \quad \quad \text{class编码方式为独热编码} $$

$$ \frac{\partial loss(\mathbf{x},\operatorname{class} )}{\partial \mathbf{x}}  = \operatorname{Softmax} (\mathbf{x}) - \boldsymbol{1}[\text { class }]$$

* Input: $(N, C)$(where $C$ = number of classes
* Target: $(N) $ where each value is $ 0 \leq \text{targets}[i] \leq C $

In [13]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [14]:
x_input = torch.arange(12, dtype=torch.float32).reshape(4, 3)
y_target = torch.tensor([1, 2, 0, 2])
x_input

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])

In [15]:
F.cross_entropy(x_input, y_target)

tensor(1.1576)

In [16]:
nn.CrossEntropyLoss()(x_input, y_target)

tensor(1.1576)

In [17]:
nn.CrossEntropyLoss(reduction='none')(x_input, y_target)

tensor([1.4076, 0.4076, 2.4076, 0.4076])

In [18]:
# target为0的损失直接为0(NLP任务中可指定ignore_index填充项)
nn.CrossEntropyLoss(reduction='none', ignore_index=0)(x_input,
                                                      y_target)  # Specifies a target value that is ignored and does not contribute to the input gradient

tensor([1.4076, 0.4076, 0.0000, 0.4076])

上式计算步骤如下

In [19]:
logsoftmax_output = F.log_softmax(x_input, dim=1)  # 注意:dim=1
logsoftmax_output

tensor([[-2.4076, -1.4076, -0.4076],
        [-2.4076, -1.4076, -0.4076],
        [-2.4076, -1.4076, -0.4076],
        [-2.4076, -1.4076, -0.4076]])

In [20]:
nlloss_output = F.nll_loss(logsoftmax_output, y_target)
print(nlloss_output)

tensor(1.1576)
